# ws 15

In [125]:
from tensorflow.keras.models import load_model

model = load_model('digitmodel.h5') # 

In [ ]:
model.summary()
model.get_config()['layers'][0]

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

f = open('digit_float.tflite', "wb")
f.write(tflite_float_model)
f.close()

float_model_size = len(tflite_float_model) / 1024
print('Float model size = %d KBs.' % float_model_size)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %d KBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

f = open('digit_quantized.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.image import rgb_to_grayscale
from PIL import Image, ImageOps

img = load_img('../digit2-1.png', target_size=(28, 28)) # 

img = ImageOps.invert(img)
img = img_to_array(img)
img = rgb_to_grayscale(img)
img = img / 255.0

test_image = np.expand_dims(img, axis=0) 

print('shape=',test_image.shape)
print('max pixel value=',np.min(img),np.max(img))
print(test_image.shape)

In [ ]:
import tensorflow as tf
from time import time

tflite_path = 'digit_float.tflite' 


interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

input = interpreter.get_input_details()
input_shape = input[0]['shape']
print('input shape:', input_shape)

input_tensor_index = input[0]["index"]
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

interpreter.set_tensor(input_tensor_index, test_image) # 

time_start = time()
interpreter.invoke()

time_end = time()
total_tflite_time = time_end - time_start
print("Total prediction time: ", total_tflite_time)

digit = np.argmax(output()[0])
print(digit)
print(output().round(2))

plt.figure(figsize=(2, 2))
plt.imshow(test_image[0], cmap=plt.cm.gray_r)
plt.title('Predicted:{}'. format(digit))
plt.xticks([]) ; plt.yticks([])
plt.show()